# Cubic Spline Interpolation

## Working Principle:
Cubic spline interpolation is a form of interpolation where the interpolant is a piecewise cubic polynomial. The goal of cubic spline interpolation is to find a smooth curve that passes through all the given data points and ensures that the first and second derivatives of the curve are continuous at each point.

## Steps for Cubic Spline Interpolation:

### 1. Set up the system of equations:
The cubic spline for each interval ([xᵢ, xᵢ₊₁]) is represented by a cubic polynomial:

**Sᵢ(x) = aᵢ(x - xᵢ)³ + bᵢ(x - xᵢ)² + cᵢ(x - xᵢ) + dᵢ**

Where (aᵢ, bᵢ, cᵢ, dᵢ) are the coefficients for the spline in the interval.

### 2. Solve for the coefficients:
The system of equations is set up by applying the following conditions:
- The spline must pass through all the data points: Sᵢ(xᵢ) = yᵢ
- The first and second derivatives must be continuous at the data points
- Boundary conditions: Typically, natural splines are used where the second derivative at the endpoints is zero

### 3. Solve the linear system:
A tridiagonal system of linear equations is formed, and solving this system yields the spline coefficients.

## Pseudocode:
**Input:** Data points (x_values, y_values)
**Output:** Spline coefficients (a, b, c, d)

1. Calculate the differences: 
   - delta_x[i] = x[i+1] - x[i]
   - delta_y[i] = y[i+1] - y[i]

2. Set up the matrix for the tridiagonal system:
   - Construct the matrix A for the system A * c = b where c contains the second derivatives at each point
   - Solve for c using a linear system solver

3. Compute the coefficients a, b, and d:
   - a[i] = y[i] (for all i)
   - b[i] = (y[i+1] - y[i]) / delta_x[i] - delta_x[i] * (2 * c[i] + c[i+1]) / 3
   - d[i] = (c[i+1] - c[i]) / (3 * delta_x[i])

4. Return the spline coefficients (a, b, c, d)

In [1]:
import numpy as np

def cubic_spline_interpolation(x, y):
    n = len(x)
    
    # Calculate differences
    delta_x = [x[i+1] - x[i] for i in range(n-1)]
    delta_y = [y[i+1] - y[i] for i in range(n-1)]
    
    # Set up tridiagonal matrix system for natural spline
    # A * c = b where c contains second derivatives
    A = np.zeros((n, n))
    b = np.zeros(n)
    
    # Natural spline boundary conditions (second derivative = 0 at endpoints)
    A[0][0] = 1
    A[n-1][n-1] = 1
    b[0] = 0
    b[n-1] = 0
    
    # Fill the tridiagonal matrix
    for i in range(1, n-1):
        A[i][i-1] = delta_x[i-1]
        A[i][i] = 2 * (delta_x[i-1] + delta_x[i])
        A[i][i+1] = delta_x[i]
        b[i] = 3 * (delta_y[i] / delta_x[i] - delta_y[i-1] / delta_x[i-1])
    
    # Solve for second derivatives
    c = np.linalg.solve(A, b)
    
    # Calculate spline coefficients
    a = y[:-1]  # a[i] = y[i]
    d = [(c[i+1] - c[i]) / (3 * delta_x[i]) for i in range(n-1)]
    b_coeff = [delta_y[i] / delta_x[i] - delta_x[i] * (2 * c[i] + c[i+1]) / 3 for i in range(n-1)]
    
    return a, b_coeff, c[:-1], d

def evaluate_spline(x_points, coeffs, x_value):
    a, b, c, d = coeffs
    n = len(x_points) - 1
    
    # Find the interval containing x_value
    for i in range(n):
        if x_points[i] <= x_value <= x_points[i+1]:
            dx = x_value - x_points[i]
            return a[i] + b[i] * dx + c[i] * dx**2 + d[i] * dx**3
    
    # If x_value is outside the range, use boundary intervals
    if x_value < x_points[0]:
        dx = x_value - x_points[0]
        return a[0] + b[0] * dx + c[0] * dx**2 + d[0] * dx**3
    else:
        dx = x_value - x_points[n-1]
        return a[n-1] + b[n-1] * dx + c[n-1] * dx**2 + d[n-1] * dx**3

# Example 1: Simple data points
print("Cubic Spline Interpolation Example 1:")
x_data = [0, 1, 2, 3]
y_data = [0, 1, 4, 9]  # roughly y = x^2

coeffs = cubic_spline_interpolation(x_data, y_data)
print(f"Spline coefficients calculated")

# Interpolate at some points
test_points = [0.5, 1.5, 2.5]
for x_val in test_points:
    y_val = evaluate_spline(x_data, coeffs, x_val)
    print(f"At x = {x_val}: y = {y_val:.4f}")

print("\n" + "="*50 + "\n")

# Example 2: More complex data
print("Cubic Spline Interpolation Example 2:")
x_data2 = [1, 2, 3, 4, 5]
y_data2 = [1, 8, 27, 64, 125]  # y = x^3

coeffs2 = cubic_spline_interpolation(x_data2, y_data2)
print(f"Spline coefficients calculated")

# Interpolate at some points
test_points2 = [1.5, 2.5, 3.5, 4.5]
for x_val in test_points2:
    y_val = evaluate_spline(x_data2, coeffs2, x_val)
    actual = x_val**3
    print(f"At x = {x_val}: y = {y_val:.4f}, actual = {actual:.4f}, error = {abs(y_val-actual):.4f}")

print("\n" + "="*50 + "\n")

# Example 3: Sin function data
import math

print("Cubic Spline Interpolation Example 3 (sin function):")
x_data3 = [0, 1, 2, 3]
y_data3 = [math.sin(x) for x in x_data3]

coeffs3 = cubic_spline_interpolation(x_data3, y_data3)
print(f"Spline coefficients calculated")

# Interpolate at some points
test_points3 = [0.5, 1.5, 2.5]
for x_val in test_points3:
    y_val = evaluate_spline(x_data3, coeffs3, x_val)
    actual = math.sin(x_val)
    print(f"At x = {x_val}: y = {y_val:.4f}, actual = {actual:.4f}, error = {abs(y_val-actual):.4f}")

Cubic Spline Interpolation Example 1:
Spline coefficients calculated
At x = 0.5: y = 0.3500
At x = 1.5: y = 2.2000
At x = 2.5: y = 6.3500


Cubic Spline Interpolation Example 2:
Spline coefficients calculated
At x = 1.5: y = 3.6161, actual = 3.3750, error = 0.2411
At x = 2.5: y = 15.6518, actual = 15.6250, error = 0.0268
At x = 3.5: y = 42.5268, actual = 42.8750, error = 0.3482
At x = 4.5: y = 92.4911, actual = 91.1250, error = 1.3661


Cubic Spline Interpolation Example 3 (sin function):
Spline coefficients calculated
At x = 0.5: y = 0.4772, actual = 0.4794, error = 0.0022
At x = 1.5: y = 0.9961, actual = 0.9975, error = 0.0014
At x = 2.5: y = 0.5895, actual = 0.5985, error = 0.0090
